---
**License**

 VexpBounce

 Tue Nov 15 12:13:14 2022\
 Copyright  2022\
 Sandro Dias Pinto Vitenti <vitenti@uel.br>

---
---

 MagDustBounce\
 Copyright (C) 2022 Sandro Dias Pinto Vitenti <vitenti@uel.br>

 numcosmo is free software: you can redistribute it and/or modify it
 under the terms of the GNU General Public License as published by the
 Free Software Foundation, either version 3 of the License, or
 (at your option) any later version.

 numcosmo is distributed in the hope that it will be useful, but
 WITHOUT ANY WARRANTY; without even the implied warranty of
 MERCHANTABILITY or FITNESS FOR A PARTICULAR PURPOSE.
 See the GNU General Public License for more details.

 You should have received a copy of the GNU General Public License along
 with this program.  If not, see <http://www.gnu.org/licenses/>.
 
---

# Magnetic field in dust dominated bouncing cosmology

In this notebook we develop the necessary objects to compute the power spectrum (and other observables) for magnetic field production in the contraction phase of a bouncing cosmology dominated by a scalar field.

The scale factor for this model is described at 1706.08830.

The electromagnetic vector field satisfy the following Hamilton equations:
\begin{equation}
\dot{A} = \frac{\Pi_A}{m}, \qquad \dot{\Pi}_A = -m\nu^2 A, \qquad m \equiv \frac{a F}{N},\quad \nu = \frac{Nk}{a},\qquad N = \frac{\mathrm{d}t}{\mathrm{d}\tau}, \qquad \alpha = \alpha_b + \frac{\tau^2}{2},\qquad \alpha = \ln(a), \qquad N = \frac{\tau}{H}.
\end{equation}
for the mode $k$. It is convenient to rewrite $F$ as
\begin{equation}
F\equiv \frac{1}{4}+B e^{-\frac{\phi^2}{\beta^2}}.
\end{equation}
The coupling is controlled by the parameters $B$ and $\beta$.

## Loading NumCosmo

The first step is to load both NumCosmo and NumCosmoMath libraries.

In [ ]:
import sys
import math
import matplotlib.pyplot as plt
import numpy as np
from matplotlib.pyplot import cm

from scipy.integrate import odeint

from numcosmo_py import Ncm, Nc
from numcosmo_py.plotting.tools import set_rc_params_article, latex_float

# Initializing the library

In [ ]:
__name__ = "NcContext"

Ncm.cfg_init()
Ncm.cfg_set_log_handler(lambda msg: sys.stdout.write(msg) and sys.stdout.flush())

# Latex definition

In [ ]:
set_rc_params_article(ncol=2)

# Model initialization

Creating a new background Vexp model

In [ ]:
Vexp_none = Nc.HICosmoVexp.new()
Vexp_none.set_em_coupling(Nc.HICosmoVexpEMCoupling.NONE)

Vexp_cauchy = Nc.HICosmoVexp.new()
Vexp_cauchy.set_em_coupling(Nc.HICosmoVexpEMCoupling.CAUCHY)

Vexp_gauss = Nc.HICosmoVexp.new()
Vexp_gauss.set_em_coupling(Nc.HICosmoVexpEMCoupling.GAUSS)

### SET 1
# current_set = "set1"
# Vexp.props.alphab   = 8.3163e-2 # Alpha (# of e-folds) at the bounce
# Vexp.props.sigmaphi = 9.0       # Width of the Gaussian solution for the WdW equation
# Vexp.props.xb       = 2.0e36    # Inverse of the scale factor at the bounce (Initial condition)


### SET 2
current_set = {
    "alphab": 7.4847e-3,  # Alpha (# of e-fold\s) at the bounce
    "sigmaphi": 100.0,  # Width of the Gaussian solution for the WdW equation
    "xb": 4.0e36,  # Inverse of the scale factor at the bounce (Initial condition)
    "dphi": -9.0e-4,  # Deviation of the Gaussian solution for the WdW equation
    "OmegaL": 1.0,  # H²(a when w=-1)/H²(a0). Basically gives the DE-dominated phase
    "Omegac": 1.0,  # Omega_d???
    "H0": 67.8,  # Hubble parameter today given by CMB observations
    "Bem": -1.0 / 4.0 + 1.0e-12,  # Amplitude of the EM gaussian coupling
    "betaem": 3.6e-1,  # Width of the EM gaussian coupling
}

Vexp_none.set_properties(**current_set)
Vexp_cauchy.set_properties(**current_set)
Vexp_gauss.set_properties(**current_set)

tc = Vexp_none.tau_xe(
    1.0e15
)  # Initial tau in expansion phase??? tau para quando a expansão é 10^15
reltol = 1.0e-14  # Relative tolerance for solving differential equations

# Model integrated time

Getting the time intervals for the computed model in terms of $\alpha \equiv \ln(a/a_0)$ and $\alpha = \alpha_b + \frac{\tau^2}{2}$.



In [ ]:
tau_min = Vexp_none.tau_min()  # Get initial time
tau_max = Vexp_none.tau_max()  # Get final time
tau_qt_c = Vexp_none.tau_qt_c()  # Get initial time of the quantum phase
tau_qt_e = Vexp_none.tau_qt_e()  # Get final time of the quantum phase
npoints = 5000

tau_a = np.linspace(tau_min, tau_max, npoints)
tau_q_a = np.linspace(tau_qt_c, tau_qt_e, npoints)

print(f"Vexp model prepared in the interval ({tau_min},{tau_max})")
print(f"Vexp quantum dominated interval ({tau_qt_c},{tau_qt_e})")

Computing background functions

In [ ]:
alpha_a = [Vexp_none.alpha(tau) for tau in tau_a]
phi_a = np.array([Vexp_none.phi(tau) for tau in tau_a])
deta_s = Ncm.SplineCubicNotaknot.new()
deta_s.set_array(tau_a, [Vexp_none.eval_nu(tau, 1.0) for tau in tau_a], True)
rescale = Vexp_none.RH_planck() / np.exp(Vexp_none.alpha_0e())
eta_a = [
    (
        -deta_s.eval_integ(tau, 0.0) * rescale
        if tau < 1.0
        else deta_s.eval_integ(0.0, tau) * rescale
    )
    for tau in tau_a
]
RS_a = np.array(
    [Vexp_none.Ricci_scale(tau) for tau in tau_a]
)  # Hubble scale goes to 0 at the bounce, but not the Ricci scalar
nu_a = np.array([Vexp_none.eval_nu(tau, 1.0) for tau in tau_a])
x_y_a = np.array([Vexp_none.x_y(tau) for tau in tau_a])

# Consistence check

Plotting $\alpha(\tau) - \alpha_b$.

In [ ]:
plt.figure()

plt.plot(tau_a, alpha_a, label=r"$\alpha$")

plt.grid(visible=True, which="both", linestyle=":", color="0.75", linewidth=0.5)
leg = plt.legend(loc="best", ncol=2, fontsize=13)

plt.xlabel(r"$\tau$")
plt.show()
pass

# Plotting scale factor as a function of tau.

$a(\tau)$

In [ ]:
plt.figure()

print(Vexp_none.alpha_0e())  # alpha_0 na expansão
print(Vexp_none.tau_xe(1.0))

plt.plot(tau_a, np.exp(alpha_a) / Vexp_none.props.xb, label=r"$a$")

plt.grid(visible=True, which="both", linestyle=":", color="0.75", linewidth=0.5)
leg = plt.legend(loc="best", ncol=2, fontsize=13)

plt.xlabel(r"$\tau$")
plt.yscale("log")
plt.show()
pass

# Plotting the scalar field as a function of tau.

$\phi(\tau)$

In [ ]:
plt.figure()

plt.plot(tau_a, phi_a, label=r"$\phi$")

plt.grid(visible=True, which="both", linestyle=":", color="0.75", linewidth=0.5)
leg = plt.legend(loc="best", ncol=2, fontsize=13)

plt.xlabel(r"$\tau$")
plt.show()

In [ ]:
plt.figure()

plt.plot(eta_a, phi_a, label=r"$\phi$")

plt.grid(visible=True, which="both", linestyle=":", color="0.75", linewidth=0.5)
leg = plt.legend(loc="best", ncol=2, fontsize=13)

plt.xscale("symlog", linthresh=1.0e-1)
plt.xlabel(r"$\eta$")
plt.show()

# Ploting Ricci scalar and mode frequency.

$R(\tau)$ and $\nu(\tau)$

In [ ]:
plt.figure()

plt.plot(tau_a, RS_a, label=r"$R_s$")
plt.plot(tau_a, nu_a, label=r"$\nu$")

plt.grid(visible=True, which="both", linestyle=":", color="0.75", linewidth=0.5)
leg = plt.legend(loc="best", ncol=2, fontsize=13)

plt.xlabel(r"$\tau$")
plt.yscale("log")

plt.show()

# Plotting background field variables

$x(\tau)$, $y(\tau)$ and $\sqrt{x^2 + y^2}$.

In [ ]:
plt.figure()

plt.plot(tau_a, x_y_a[:, 0], label="$x$")
plt.plot(tau_a, x_y_a[:, 1], label="$y$")
plt.plot(tau_a, np.hypot(x_y_a[:, 0], x_y_a[:, 1]), label="$\sqrt{x^2+y^2}$")

plt.grid(visible=True, which="both", linestyle=":", color="0.75", linewidth=0.5)
leg = plt.legend(loc="best", ncol=2, fontsize=13)

plt.xlabel(r"$\tau$")
plt.yscale("symlog")

plt.show()

# Close-up on the Friedmann constraint in the quantum regime
plt.figure()

plt.plot(
    tau_a,
    np.sqrt(x_y_a[:, 0] * x_y_a[:, 0] + x_y_a[:, 1] * x_y_a[:, 1]),
    label="$x^2+y^2$",
)
plt.plot(
    tau_a,
    np.hypot(x_y_a[:, 0], x_y_a[:, 1]),
    label="$\sqrt{x^2+y^2}$",
    linestyle="dashed",
)

plt.xlim(-0.25, 0.25)
plt.yscale("symlog")

plt.show()

# Plotting magnetic field background quantities 

$\phi(\tau)$, $f(\phi(\tau)) = e^{-\phi^2/\beta^2}$ and the equation of state $w$.

In [ ]:
plt.figure()

plt.plot(tau_a, phi_a, label="$\phi$")
# plt.plot (tau_a, np.exp (-(phi_a / beta)**2), label=r'$f(\phi)$')
plt.plot(tau_a, 2.0 * x_y_a[:, 0] ** 2 - 1.0, label="$w = 2x^2-1$")

plt.grid(visible=True, which="both", linestyle=":", color="0.75", linewidth=0.5)
leg = plt.legend(loc="best", ncol=2, fontsize=13)

plt.xlabel(r"$\tau$")
# plt.yscale('symlog')
plt.xlim(-2.0, tau_qt_e)
plt.ylim(-1.1, 1.1)

plt.show()

###
### Equation of state
###

plt.figure()

plt.plot(tau_a, 2.0 * x_y_a[:, 0] ** 2 - 1.0, label="$w = 2x^2-1$")

plt.grid(visible=True, which="both", linestyle=":", color="0.75", linewidth=0.5)
leg = plt.legend(loc="best", ncol=2, fontsize=13)
plt.xlim(-0.025, 0.025)
plt.xlabel(r"$\tau$")

plt.show()

# Defines the electromagnetic field model

This model use the Complex Structure Quantization method CSQ1D.

In [ ]:
pem_none = Nc.HIPertEM.new()
pem_none.set_abstol(
    1.0e-200
)  # The vacuum is a fixed point at zero, we need an absolute tolerance to evolve the system

pem_cauchy = Nc.HIPertEM.new()

pem_gauss = Nc.HIPertEM.new()
pem_gauss.set_initial_condition_type(Ncm.CSQ1DInitialStateType.ADIABATIC2)

pem_none.set_k(1.0e3)
pem_cauchy.set_k(1.0e3)
pem_gauss.set_k(1.0e3)

# Setting up models

Setting up all models, including the benchmark with no coupling.

# Finding starting point as the end of the adiabatic phase

Setting limit for the adiabatic phase with $10^{-9}$.

In [ ]:
print(f"Searching for WKB scale in ({tau_min}, {tau_qt_c})")

(Found_tau_i_cauchy, tau_i_cauchy) = pem_cauchy.find_adiab_time_limit(
    Vexp_cauchy, tau_min, -1.0e-2, 1.0e-9
)
(Found_tau_i_gauss, tau_i_gauss) = pem_gauss.find_adiab_time_limit(
    Vexp_gauss, tau_min, -1.0e-2, 1.0e-9
)

# The vacuum for no coupling is a fixed point, all points are adiabatic, using the same interval as Cauchy
Found_tau_i_none, tau_i_none = Found_tau_i_cauchy, tau_i_cauchy

# Checking if the search for the starting time was successful 

In [ ]:
tau_f = 10.0

print(
    f"Found ({Found_tau_i_none  }) WKB scale for no-potential at     {tau_i_none:7.3f}"
)
print(
    f"Found ({Found_tau_i_cauchy}) WKB scale for cauchy-potential at {tau_i_cauchy:7.3f} (B = {Vexp_gauss.props.Bem:.2e}, beta = {Vexp_gauss.props.betaem})"
)
print(
    f"Found ({Found_tau_i_gauss }) WKB scale for gauss-potential at  {tau_i_gauss:7.3f} (B = {Vexp_gauss.props.Bem:.2e}, beta = {Vexp_gauss.props.betaem})"
)

print(pem_none.eval_F1(Vexp_none, tau_i_none))
print(pem_none.eval_F2(Vexp_none, tau_i_none))
print(pem_cauchy.eval_F1(Vexp_cauchy, tau_i_cauchy))
print(pem_cauchy.eval_F2(Vexp_cauchy, tau_i_cauchy))
print(pem_gauss.eval_F1(Vexp_gauss, tau_i_gauss))
print(pem_gauss.eval_F2(Vexp_gauss, tau_i_gauss))

state_none = Ncm.CSQ1DState.new()
state_cauchy = Ncm.CSQ1DState.new()
state_gauss = Ncm.CSQ1DState.new()

print(pem_none.compute_adiab(Vexp_none, tau_i_none, state_none)[0].get_ag())
print(pem_cauchy.compute_adiab(Vexp_cauchy, tau_i_cauchy, state_cauchy)[0].get_ag())
print(pem_gauss.compute_adiab(Vexp_gauss, tau_i_gauss, state_gauss)[0].get_ag())

In [ ]:
plt.figure()

tau_ad_a = np.linspace(-2.35, -1.0, 100)
tau_ad_a = tau_a

F1_a_none = np.array([pem_none.eval_F1(Vexp_none, tau) for tau in tau_ad_a])
F2_a_none = np.array([pem_none.eval_F2(Vexp_none, tau) for tau in tau_ad_a])
nu_a_none = np.array([pem_none.eval_nu(Vexp_none, tau) for tau in tau_ad_a])

F1_a_cauchy = np.array([pem_cauchy.eval_F1(Vexp_cauchy, tau) for tau in tau_ad_a])
F2_a_cauchy = np.array([pem_cauchy.eval_F2(Vexp_cauchy, tau) for tau in tau_ad_a])
nu_a_cauchy = np.array([pem_cauchy.eval_nu(Vexp_cauchy, tau) for tau in tau_ad_a])

F1_a_gauss = np.array([pem_gauss.eval_F1(Vexp_gauss, tau) for tau in tau_ad_a])
F2_a_gauss = np.array([pem_gauss.eval_F2(Vexp_gauss, tau) for tau in tau_ad_a])
nu_a_gauss = np.array([pem_gauss.eval_nu(Vexp_gauss, tau) for tau in tau_ad_a])

plt.plot(tau_ad_a, F1_a_none, label=r"no-pot $F_1$")
plt.plot(tau_ad_a, F1_a_cauchy, label=r"cauchy $F_1$")
plt.plot(tau_ad_a, F1_a_gauss, label=r"gauss  $F_1$")

plt.plot(tau_ad_a, F2_a_none, label=r"no-pot $F_2$")
plt.plot(tau_ad_a, F2_a_cauchy, label=r"cauchy $F_2$")
plt.plot(tau_ad_a, F2_a_gauss, label=r"gauss  $F_2$")

plt.grid(visible=True, which="both", linestyle=":", color="0.75", linewidth=0.5)
leg = plt.legend(loc="best", ncol=2, fontsize=13)

plt.xlabel(r"$\tau$")
plt.xscale("symlog")
plt.yscale("symlog", linthresh=1.0e-20)

plt.show()

tau_i = -0.94
tau_f = 5.0
print(csq1d0.eval_adiab_at (Vexp, tau_i))
print(csq1d1.eval_adiab_at (Vexp, tau_i))

In [ ]:
pem_none.set_init_cond_adiab(Vexp_none, tau_i_none)
pem_cauchy.set_init_cond_adiab(Vexp_cauchy, tau_i_cauchy)
pem_gauss.set_init_cond_adiab(Vexp_gauss, tau_i_gauss)

In [ ]:
pem_none.set_ti(tau_i_none)
pem_none.set_tf(tau_f)
pem_none.prepare(Vexp_none)

pem_cauchy.set_ti(tau_i_cauchy)
pem_cauchy.set_tf(tau_f)
pem_cauchy.prepare(Vexp_cauchy)

pem_gauss.set_ti(tau_i_gauss)
pem_gauss.set_tf(tau_f)
pem_gauss.prepare(Vexp_gauss)

tau_evol_a_none, tau_s_none = pem_none.get_time_array()
tau_evol_a_none = np.array(tau_evol_a_none)

tau_evol_a_cauchy, tau_s_gauss = pem_cauchy.get_time_array()
tau_evol_a_cauchy = np.array(tau_evol_a_cauchy)

tau_evol_a_gauss, tau_s_gauss = pem_gauss.get_time_array()
tau_evol_a_gauss = np.array(tau_evol_a_gauss)

In [ ]:
PB_a_none = []
PE_a_none = []
J11_a_none = []
J22_a_none = []

PB_a_cauchy = []
PE_a_cauchy = []
J11_a_cauchy = []
J22_a_cauchy = []

PB_a_gauss = []
PE_a_gauss = []
J11_a_gauss = []
J22_a_gauss = []

for tau in tau_evol_a_none:
    PE, PB = pem_none.eval_PE_PB(Vexp_none, tau)
    J11, _, J22 = pem_none.eval_at(tau, state_none).get_J()
    PE_a_none.append(PE)
    PB_a_none.append(PB)
    J11_a_none.append(J11)
    J22_a_none.append(J22)

for tau in tau_evol_a_cauchy:
    PE, PB = pem_cauchy.eval_PE_PB(Vexp_cauchy, tau)
    J11, _, J22 = pem_cauchy.eval_at(tau, state_cauchy).get_J()
    PE_a_cauchy.append(PE)
    PB_a_cauchy.append(PB)
    J11_a_cauchy.append(J11)
    J22_a_cauchy.append(J22)

for tau in tau_evol_a_gauss:
    PE, PB = pem_gauss.eval_PE_PB(Vexp_gauss, tau)
    J11, _, J22 = pem_gauss.eval_at(tau, state_gauss).get_J()
    PE_a_gauss.append(PE)
    PB_a_gauss.append(PB)
    J11_a_gauss.append(J11)
    J22_a_gauss.append(J22)

PB_a_none = np.array(PB_a_none)
PE_a_none = np.array(PE_a_none)
J11_a_none = np.array(J11_a_none)
J22_a_none = np.array(J22_a_none)

PB_a_cauchy = np.array(PB_a_cauchy)
PE_a_cauchy = np.array(PE_a_cauchy)
J11_a_cauchy = np.array(J11_a_cauchy)
J22_a_cauchy = np.array(J22_a_cauchy)

PB_a_gauss = np.array(PB_a_gauss)
PE_a_gauss = np.array(PE_a_gauss)
J11_a_gauss = np.array(J11_a_gauss)
J22_a_gauss = np.array(J22_a_gauss)

In [ ]:
mylw = 1

plt.figure(figsize=(16, 8))

plt.plot(
    tau_evol_a_none,
    J22_a_none,
    lw=mylw,
    label=f"no-pot $J_{{22}}(k_s = {pem_none.get_k()})$",
)
plt.plot(
    tau_evol_a_gauss,
    J22_a_gauss,
    lw=mylw,
    label=f"gauss $J_{{22}}(k_s = {pem_gauss.get_k()})$",
)

leg = plt.legend(loc="best", ncol=2, fontsize=13)
plt.yscale("log")
plt.show()

plt.figure(figsize=(16, 8))

plt.plot(
    tau_evol_a_cauchy,
    J22_a_cauchy,
    lw=mylw,
    label=f"cauchy $J_{{22}}(k_s = {pem_cauchy.get_k()})$",
)

leg = plt.legend(loc="best", ncol=2, fontsize=13)
plt.yscale("log")
plt.show()

# Loop over k (TO FIX: same J for k=1 and k=4.000)

# plt.figure (figsize=(16, 8))

# mylw = 1

# for k in k_a:

#     plt.plot (tau_evol_a0, J22_a0, lw=mylw, label = f'$J^0_{{22}}(k_s = %s)$'% latex_float (k))
#     plt.plot (tau_evol_a1, J22_a1, lw=mylw, label = f'$J^1_{{22}}(k_s = %s)$'% latex_float (k))

# leg = plt.legend (loc="best", ncol = 2, fontsize=13)
# plt.yscale("log")

In [ ]:
plt.figure(figsize=(14, 7))

plt.plot(
    tau_evol_a_none,
    J11_a_none,
    lw=mylw,
    label=f"no-pot $J_{{11}}(k_s = {pem_none.get_k()})$",
)
plt.plot(
    tau_evol_a_gauss,
    J11_a_gauss,
    lw=mylw,
    label=f"gauss $J_{{11}}(k_s = {pem_gauss.get_k()})$",
)

leg = plt.legend(loc="best", ncol=2, fontsize=13)
plt.yscale("log")
plt.show()

plt.figure(figsize=(14, 7))

plt.plot(
    tau_evol_a_cauchy,
    J11_a_cauchy,
    lw=mylw,
    label=f"cauchy $J_{{11}}(k_s = {pem_cauchy.get_k()})$",
)

leg = plt.legend(loc="best", ncol=2, fontsize=13)
plt.yscale("log")
plt.show()

In [ ]:
plt.figure(figsize=(14, 7))

plt.plot(
    tau_evol_a_none,
    PB_a_none,
    lw=mylw,
    label=f"no-pot $P_{{B}}(k_s = {pem_none.get_k()})$",
)
plt.plot(
    tau_evol_a_gauss,
    PB_a_gauss,
    lw=mylw,
    label=f"gauss $P_{{B}}(k_s = {pem_gauss.get_k()})$",
)
plt.plot(
    tau_evol_a_cauchy,
    PB_a_cauchy,
    lw=mylw,
    label=f"cauchy $P_{{B}}(k_s = {pem_cauchy.get_k()})$",
)

leg = plt.legend(loc="best", ncol=2, fontsize=13)
plt.yscale("log")
# plt.xlim(8.0, None)
# plt.ylim(1.0e-60,None)
plt.show()

plt.figure(figsize=(14, 7))

plt.plot(
    tau_evol_a_none,
    PE_a_none,
    lw=mylw,
    label=f"no-pot $P_{{E}}(k_s = {pem_none.get_k()})$",
)
plt.plot(
    tau_evol_a_gauss,
    PE_a_gauss,
    lw=mylw,
    label=f"gauss $P_{{E}}(k_s = {pem_gauss.get_k()})$",
)
plt.plot(
    tau_evol_a_cauchy,
    PE_a_cauchy,
    lw=mylw,
    label=f"cauchy $P_{{E}}(k_s = {pem_cauchy.get_k()})$",
)

leg = plt.legend(loc="best", ncol=2, fontsize=13)
plt.yscale("log")
# plt.xlim(-5.0,5.0)
# plt.ylim(1.0e-85, None)

plt.show()

In [ ]:
plt.figure(figsize=(14, 7))

plt.plot(
    tau_evol_a_none,
    PB_a_none,
    lw=mylw,
    label=f"no-pot $P_{{B}}(k_s = {pem_none.get_k()})$",
)
# plt.plot(tau_evol_a_gauss, PB_a_gauss, lw=mylw, label=f"gauss $P_{{B}}(k_s = {pem_gauss.get_k()})$")
# plt.plot(tau_evol_a_cauchy, PB_a_cauchy, lw=mylw, label=f"cauchy $P_{{B}}(k_s = {pem_cauchy.get_k()})$")
plt.plot(
    tau_evol_a_none,
    PE_a_none,
    lw=mylw,
    label=f"no-pot $P_{{E}}(k_s = {pem_none.get_k()})$",
)
# plt.plot(tau_evol_a_gauss, PE_a_gauss, lw=mylw, label=f"gauss $P_{{E}}(k_s = {pem_gauss.get_k()})$")
# plt.plot(tau_evol_a_cauchy, PE_a_cauchy, lw=mylw, label=f"cauchy $P_{{E}}(k_s = {pem_cauchy.get_k()})$")

leg = plt.legend(loc="best", ncol=2, fontsize=13)
plt.yscale("log")
# plt.xlim(-5.0,5.0)
# plt.ylim(1.0e-125, None)

plt.show()

In [ ]:
###
### Loop over k
###

tau_ad_a = np.linspace(-1.35, -1.0, 100)

ki = 1.0
kf = 4.0e3
k_a = np.geomspace(ki, kf, 3)

B0 = 1.0e-50
beta0 = 8.0e-1
B1 = 1.0e50
beta1 = 8.0e-1


csq1d_nopot = PyCSQ1DMagCauchy(B0, beta0)
csq1d_gauss = PyCSQ1DMagGauss(B1, beta1)
csq1d_cauchy = PyCSQ1DMagCauchy(B1, beta1)


# Define map colours for easier visualisation
# Maps are defined there: https://matplotlib.org/stable/tutorials/colors/colormaps.html
# colormapF1a_nopot  = plt.get_cmap('Blues')
# colorsF1a_nopot    = iter(colormapF1a_nopot(np.linspace(0,1,len(k_a))))
# colormapF1a_gauss  = plt.get_cmap('Greens')
# colorsF1a_gauss    = iter(colormapF1a_gauss(np.linspace(0,1,len(k_a))))
# colormapF1a_cauchy = plt.get_cmap('Reds')
# colorsF1a_cauchy   = iter(colormapF1a_cauchy(np.linspace(0,1,len(k_a))))

# colormapF2a_nopot  = plt.get_cmap('Dark2')
# colorsF2a_nopot    = iter(colormapF2a_nopot(np.linspace(0,1,len(k_a))))
# colormapF2a_gauss  = plt.get_cmap('Dark2')
# colorsF2a_gauss    = iter(colormapF2a_gauss(np.linspace(0,1,len(k_a))))
# colormapF2a_cauchy = plt.get_cmap('Dark2')
# colorsF2a_cauchy   = iter(colormapF2a_cauchy(np.linspace(0,1,len(k_a))))


color_nopot = iter(
    cm.Reds(np.linspace(1 / 2, 1, len(k_a)))
)  # Go from 1/2 to 1 to display colours not too bright
color_gauss = iter(cm.Blues(np.linspace(1 / 2, 1, len(k_a))))
color_cauchy = iter(cm.Greens(np.linspace(1 / 2, 1, len(k_a))))

plt.figure(figsize=(16, 8))


###
### TO DO: fix the overlapping ticks on the y-axis
###

# First loop for F1
for k in k_a:
    csq1d_nopot.set_k(k)
    csq1d_gauss.set_k(k)
    csq1d_cauchy.set_k(k)

    print(f"Searching for WKB scale in ({tau_min}, {tau_qt_c})")

    (Found_tau_i_nopot, tau_i_nopot) = csq1d_nopot.find_adiab_time_limit(
        Vexp, tau_min, -1.0e-2, 1.0e-9
    )
    (Found_tau_i_gauss, tau_i_gauss) = csq1d_gauss.find_adiab_time_limit(
        Vexp, tau_min, -1.0e-2, 1.0e-9
    )
    (Found_tau_i_cauchy, tau_i_cauchy) = csq1d_cauchy.find_adiab_time_limit(
        Vexp, tau_min, -1.0e-2, 1.0e-9
    )

    print(csq1d_nopot.eval_F1(Vexp, tau_i_nopot, k))
    print(csq1d_gauss.eval_F1(Vexp, tau_i_gauss, k))
    print(csq1d_cauchy.eval_F1(Vexp, tau_i_cauchy, k))

    print(csq1d_nopot.eval_adiab_at(Vexp, tau_i_nopot))
    print(csq1d_gauss.eval_adiab_at(Vexp, tau_i_gauss))
    print(csq1d_cauchy.eval_adiab_at(Vexp, tau_i_cauchy))

    F1_a_nopot = np.array([csq1d_nopot.eval_F1(Vexp, tau, k) for tau in tau_ad_a])
    # F2_a_nopot = np.array ([csq1d_nopot.eval_F2 (Vexp, tau, k) for tau in tau_ad_a])
    nu_a_nopot = np.array([csq1d_nopot.eval_nu(Vexp, tau, k) for tau in tau_ad_a])

    F1_a_gauss = np.array([csq1d_gauss.eval_F1(Vexp, tau, k) for tau in tau_ad_a])
    # F2_a_gauss = np.array ([csq1d_gauss.eval_F2 (Vexp, tau, k) for tau in tau_ad_a])
    nu_a_gauss = np.array([csq1d_gauss.eval_nu(Vexp, tau, k) for tau in tau_ad_a])

    F1_a_cauchy = np.array([csq1d_cauchy.eval_F1(Vexp, tau, k) for tau in tau_ad_a])
    # F2_a_cauchy = np.array ([csq1d_cauchy.eval_F2 (Vexp, tau, k) for tau in tau_ad_a])
    nu_a_cauchy = np.array([csq1d_cauchy.eval_nu(Vexp, tau, k) for tau in tau_ad_a])

    c_nopot = next(color_nopot)
    c_gauss = next(color_gauss)
    c_cauchy = next(color_cauchy)

    plt.plot(
        tau_ad_a,
        F1_a_nopot,
        label=r"$F^{nopot}_1$  (k = $%s$)" % latex_float(k),
        c=c_nopot,
    )  # color=next(colorsF1a_nopot))
    plt.plot(
        tau_ad_a,
        F1_a_gauss,
        label=r"$F^{gauss}_1$  (k = $%s$)" % latex_float(k),
        c=c_gauss,
    )  # color=next(colorsF1a_gauss))
    plt.plot(
        tau_ad_a,
        F1_a_cauchy,
        label=r"$F^{cauchy}_1$ (k = $%s$)" % latex_float(k),
        c=c_cauchy,
    )  # color=next(colorsF1a_cauchy))
    # plt.plot (tau_ad_a, F2_a_nopot,  label=r"$F^{nopot}_2$  (k = $%s$)"% latex_float (k),c=c1)#color=next(colorsF2a_nopot))
    # plt.plot (tau_ad_a, F2_a_gauss,  label=r"$F^{gauss}_2$  (k = $%s$)"% latex_float (k),c=c2)#color=next(colorsF2a_gauss))
    # plt.plot (tau_ad_a, F2_a_cauchy, label=r"$F^{cauchy}_2$ (k = $%s$)"% latex_float (k),c=c3)#color=next(colorsF2a_cauchy))


plt.grid(visible=True, which="both", linestyle=":", color="0.75", linewidth=0.5)
leg = plt.legend(loc="best", ncol=2, fontsize=16)

plt.xlabel(r"$\tau$")
plt.xscale("symlog")
plt.yscale("symlog", linthresh=1.0e-10)

# Erase overlapping ticks labels
ticks_F1 = plt.yticks()[0]
ticks_F1[2] = 0
ticks_F1[4] = 0
plt.yticks(ticks_F1)

plt.savefig(plotpath + "plotF1_" + current_set + ".pdf")

plt.show()


# Reinitialise the color map
color_nopot = iter(cm.Reds(np.linspace(1 / 2, 1, len(k_a))))
color_gauss = iter(cm.Blues(np.linspace(1 / 2, 1, len(k_a))))
color_cauchy = iter(cm.Greens(np.linspace(1 / 2, 1, len(k_a))))

plt.figure(figsize=(16, 8))

# Second loop for F2
for k in k_a:
    csq1d_nopot.set_k(k)
    csq1d_gauss.set_k(k)
    csq1d_cauchy.set_k(k)

    print(f"Searching for WKB scale in ({tau_min}, {tau_qt_c})")

    (Found_tau_i_nopot, tau_i_nopot) = csq1d_nopot.find_adiab_time_limit(
        Vexp, tau_min, -1.0e-2, 1.0e-9
    )
    (Found_tau_i_gauss, tau_i_gauss) = csq1d_gauss.find_adiab_time_limit(
        Vexp, tau_min, -1.0e-2, 1.0e-9
    )
    (Found_tau_i_cauchy, tau_i_cauchy) = csq1d_cauchy.find_adiab_time_limit(
        Vexp, tau_min, -1.0e-2, 1.0e-9
    )

    print(csq1d_nopot.eval_F1(Vexp, tau_i_nopot, k))
    print(csq1d_gauss.eval_F1(Vexp, tau_i_gauss, k))
    print(csq1d_cauchy.eval_F1(Vexp, tau_i_cauchy, k))

    print(csq1d_nopot.eval_adiab_at(Vexp, tau_i_nopot))
    print(csq1d_gauss.eval_adiab_at(Vexp, tau_i_gauss))
    print(csq1d_cauchy.eval_adiab_at(Vexp, tau_i_cauchy))

    # F1_a_nopot = np.array ([csq1d_nopot.eval_F1 (Vexp, tau, k) for tau in tau_ad_a])
    F2_a_nopot = np.array([csq1d_nopot.eval_F2(Vexp, tau, k) for tau in tau_ad_a])
    nu_a_nopot = np.array([csq1d_nopot.eval_nu(Vexp, tau, k) for tau in tau_ad_a])

    # F1_a_gauss = np.array ([csq1d_gauss.eval_F1 (Vexp, tau, k) for tau in tau_ad_a])
    F2_a_gauss = np.array([csq1d_gauss.eval_F2(Vexp, tau, k) for tau in tau_ad_a])
    nu_a_gauss = np.array([csq1d_gauss.eval_nu(Vexp, tau, k) for tau in tau_ad_a])

    # F1_a_cauchy = np.array ([csq1d_cauchy.eval_F1 (Vexp, tau, k) for tau in tau_ad_a])
    F2_a_cauchy = np.array([csq1d_cauchy.eval_F2(Vexp, tau, k) for tau in tau_ad_a])
    nu_a_cauchy = np.array([csq1d_cauchy.eval_nu(Vexp, tau, k) for tau in tau_ad_a])

    c_nopot = next(color_nopot)
    c_gauss = next(color_gauss)
    c_cauchy = next(color_cauchy)

    # plt.plot (tau_ad_a, F1_a_nopot,  label=r"$F^{nopot}_1$  (k = $%s$)"% latex_float (k),c=c1)#color=next(colorsF1a_nopot))
    # plt.plot (tau_ad_a, F1_a_gauss,  label=r"$F^{gauss}_1$  (k = $%s$)"% latex_float (k),c=c2)#color=next(colorsF1a_gauss))
    # plt.plot (tau_ad_a, F1_a_cauchy, label=r"$F^{cauchy}_1$ (k = $%s$)"% latex_float (k),c=c3)#color=next(colorsF1a_cauchy))
    plt.plot(
        tau_ad_a,
        F2_a_nopot,
        label=r"$F^{nopot}_2$  (k = $%s$)" % latex_float(k),
        c=c_nopot,
    )  # color=next(colorsF2a_nopot))
    plt.plot(
        tau_ad_a,
        F2_a_gauss,
        label=r"$F^{gauss}_2$  (k = $%s$)" % latex_float(k),
        c=c_gauss,
    )  # color=next(colorsF2a_gauss))
    plt.plot(
        tau_ad_a,
        F2_a_cauchy,
        label=r"$F^{cauchy}_2$ (k = $%s$)" % latex_float(k),
        c=c_cauchy,
    )  # color=next(colorsF2a_cauchy))

plt.grid(visible=True, which="both", linestyle=":", color="0.75", linewidth=0.5)
leg = plt.legend(loc="best", ncol=2, fontsize=16)

plt.xlabel(r"$\tau$")
plt.xscale("symlog")
plt.yscale("symlog", linthresh=1.0e-10)

# Erase overlapping ticks labels
ticks_F2 = plt.yticks()[0]
ticks_F2[7] = 0
ticks_F2[9] = 0
plt.yticks(ticks_F2)

plt.savefig(plotpath + "plotF2_" + current_set + ".pdf")

plt.show()

In [ ]:
ki = 1.0
kf = 4.0e3
k_a = np.geomspace(ki, kf, 3)

tau_f = 10.0

B0 = 1.0e-50
beta0 = 8.0e-1
B1 = 1.0e50
beta1 = 8.0e-1


csq1d_nopot = PyCSQ1DMagCauchy(B0, beta0)
csq1d_gauss = PyCSQ1DMagGauss(B1, beta1)
csq1d_cauchy = PyCSQ1DMagCauchy(B1, beta1)

# Define map colours for easier visualisation
# Maps are defined there: https://matplotlib.org/stable/tutorials/colors/colormaps.html
# colormapF1a0= plt.get_cmap('Accent')
# colorsF1a0 = iter(colormapF1a0(np.linspace(0,1,10)))
# colormapF1a1= plt.get_cmap('Accent')
# colorsF1a1 = iter(colormapF1a1(np.linspace(0,1,10)))
# colormapF2a0= plt.get_cmap('Dark2')
# colorsF2a0 = iter(colormapF2a0(np.linspace(0,1,10)))
# colormapF2a1= plt.get_cmap('Dark2')
# colorsF2a1 = iter(colormapF2a1(np.linspace(0,1,10)))


color_nopot = iter(
    cm.Reds(np.linspace(1 / 2, 1, len(k_a)))
)  # Go from 1/2 to 1 to display colours not too bright
color_gauss = iter(cm.Blues(np.linspace(1 / 2, 1, len(k_a))))
color_cauchy = iter(cm.Greens(np.linspace(1 / 2, 1, len(k_a))))

tau_ad_a = np.linspace(-1.35, -1.0, 100)
tau_ad_a = tau_a

plt.figure(figsize=(14, 7))

mylw = 1

for k in k_a:
    csq1d_nopot.set_k(k)
    csq1d_gauss.set_k(k)
    csq1d_cauchy.set_k(k)

    print(f"Searching for WKB scale in ({tau_min}, {tau_qt_c})")

    (Found_tau_i_nopot, tau_i_nopot) = csq1d_nopot.find_adiab_time_limit(
        Vexp, tau_min, -1.0e-2, 1.0e-9
    )
    (Found_tau_i_gauss, tau_i_gauss) = csq1d_gauss.find_adiab_time_limit(
        Vexp, tau_min, -1.0e-2, 1.0e-9
    )
    (Found_tau_i_cauchy, tau_i_cauchy) = csq1d_cauchy.find_adiab_time_limit(
        Vexp, tau_min, -1.0e-2, 1.0e-9
    )

    print(csq1d_nopot.eval_F1(Vexp, tau_i_nopot, k))
    print(csq1d_gauss.eval_F1(Vexp, tau_i_gauss, k))
    print(csq1d_cauchy.eval_F1(Vexp, tau_i_cauchy, k))

    print(csq1d_nopot.eval_adiab_at(Vexp, tau_i_nopot))
    print(csq1d_gauss.eval_adiab_at(Vexp, tau_i_gauss))
    print(csq1d_cauchy.eval_adiab_at(Vexp, tau_i_cauchy))

    F1_a_nopot = np.array([csq1d_nopot.eval_F1(Vexp, tau, k) for tau in tau_ad_a])
    F2_a_nopot = np.array([csq1d_nopot.eval_F2(Vexp, tau, k) for tau in tau_ad_a])
    nu_a_nopot = np.array([csq1d_nopot.eval_nu(Vexp, tau, k) for tau in tau_ad_a])

    F1_a_gauss = np.array([csq1d_gauss.eval_F1(Vexp, tau, k) for tau in tau_ad_a])
    F2_a_gauss = np.array([csq1d_gauss.eval_F2(Vexp, tau, k) for tau in tau_ad_a])
    nu_a_gauss = np.array([csq1d_gauss.eval_nu(Vexp, tau, k) for tau in tau_ad_a])

    F1_a_cauchy = np.array([csq1d_cauchy.eval_F1(Vexp, tau, k) for tau in tau_ad_a])
    F2_a_cauchy = np.array([csq1d_cauchy.eval_F2(Vexp, tau, k) for tau in tau_ad_a])
    nu_a_cauchy = np.array([csq1d_cauchy.eval_nu(Vexp, tau, k) for tau in tau_ad_a])

    csq1d_nopot.set_init_cond_adiab(Vexp, tau_i_nopot)
    csq1d_gauss.set_init_cond_adiab(Vexp, tau_i_gauss)
    csq1d_cauchy.set_init_cond_adiab(Vexp, tau_i_cauchy)

    csq1d_nopot.set_ti(tau_i_nopot)
    csq1d_nopot.set_tf(tau_f)
    csq1d_nopot.prepare(Vexp)

    csq1d_gauss.set_ti(tau_i_gauss)
    csq1d_gauss.set_tf(tau_f)
    csq1d_gauss.prepare(Vexp)

    csq1d_cauchy.set_ti(tau_i_cauchy)
    csq1d_cauchy.set_tf(tau_f)
    csq1d_cauchy.prepare(Vexp)

    tau_evol_a_nopot, tau_s_nopot = csq1d_nopot.get_time_array()
    tau_evol_a_nopot = np.array(tau_evol_a_nopot)

    tau_evol_a_gauss, tau_s_gauss = csq1d_gauss.get_time_array()
    tau_evol_a_gauss = np.array(tau_evol_a_gauss)

    tau_evol_a_cauchy, tau_s_cauchy = csq1d_cauchy.get_time_array()
    tau_evol_a_cauchy = np.array(tau_evol_a_cauchy)

    PB_a_nopot = []
    PE_a_nopot = []
    J11_a_nopot = []
    J22_a_nopot = []

    PB_a_gauss = []
    PE_a_gauss = []
    J11_a_gauss = []
    J22_a_gauss = []

    PB_a_cauchy = []
    PE_a_cauchy = []
    J11_a_cauchy = []
    J22_a_cauchy = []

    for tau in tau_evol_a_nopot:
        PB, PE, J11, J22 = csq1d_nopot.eval_PB_PE(Vexp, tau)
        PB_a_nopot.append(PB)
        PE_a_nopot.append(PE)
        J11_a_nopot.append(J11)
        J22_a_nopot.append(J22)

    for tau in tau_evol_a_gauss:
        PB, PE, J11, J22 = csq1d_gauss.eval_PB_PE(Vexp, tau)
        #         PB_a_gauss.append(PB)
        #         PE_a_gauss.append(PE)
        #         J11_a_gauss.append(J11)
        J22_a_gauss.append(J22)

    for tau in tau_evol_a_cauchy:
        PB, PE, J11, J22 = csq1d_cauchy.eval_PB_PE(Vexp, tau)
        #         PB_a_cauchy.append(PB)
        #         PE_a_cauchy.append(PE)
        #         J11_a_cauchy.append(J11)
        J22_a_cauchy.append(J22)

    #     PB_a_nopot = np.array(PB_a_nopot)
    #     PE_a_nopot = np.array(PE_a_nopot)
    #     J11_a_nopot = np.array(J11_a_nopot)
    J22_a_nopot = np.array(J22_a_nopot)

    #     PB_a_gauss = np.array(PB_a_gauss)
    #     PE_a_gauss = np.array(PE_a_gauss)
    #     J11_a_gauss = np.array(J11_a_gauss)
    J22_a_gauss = np.array(J22_a_gauss)

    #     PB_a_cauchy = np.array(PB_a_cauchy)
    #     PE_a_cauchy = np.array(PE_a_cauchy)
    #     J11_a_cauchy = np.array(J11_a_cauchy)
    J22_a_cauchy = np.array(J22_a_cauchy)

    c_nopot = next(color_nopot)
    c_gauss = next(color_gauss)
    c_cauchy = next(color_cauchy)

    plt.plot(
        tau_evol_a_nopot,
        J22_a_nopot,
        c=c_nopot,
        lw=mylw,
        label=f"$J^{{nopot}}_{{22}}$ (k = $%s$)" % latex_float(k),
    )
    plt.plot(
        tau_evol_a_gauss,
        J22_a_gauss,
        c=c_gauss,
        lw=mylw,
        label=f"$J^{{gauss}}_{{22}}$ (k = $%s$)" % latex_float(k),
    )
    plt.plot(
        tau_evol_a_cauchy,
        J22_a_cauchy,
        c=c_cauchy,
        lw=mylw,
        label=f"$J^{{cauchy}}_{{22}}$ (k = $%s$)" % latex_float(k),
    )


leg = plt.legend(loc="center left", ncol=2, fontsize=16)
plt.yscale("log")
plt.xlabel(r"$\tau$")

plt.savefig(plotpath + "plotJ22_" + current_set + ".pdf")

plt.show()

In [ ]:
ki = 1.0
kf = 4.0e3
k_a = np.geomspace(ki, kf, 3)

B0 = 1.0e-50
beta0 = 8.0e-1
B1 = 1.0e50
beta1 = 8.0e-1

csq1d_nopot = PyCSQ1DMagCauchy(B0, beta0)
csq1d_gauss = PyCSQ1DMagGauss(B1, beta1)
csq1d_cauchy = PyCSQ1DMagCauchy(B1, beta1)

# Define map colours for easier visualisation
# Maps are defined there: https://matplotlib.org/stable/tutorials/colors/colormaps.html
# colormapF1a0= plt.get_cmap('Accent')
# colorsF1a0 = iter(colormapF1a0(np.linspace(0,1,10)))
# colormapF1a1= plt.get_cmap('Accent')
# colorsF1a1 = iter(colormapF1a1(np.linspace(0,1,10)))
# colormapF2a0= plt.get_cmap('Dark2')
# colorsF2a0 = iter(colormapF2a0(np.linspace(0,1,10)))
# colormapF2a1= plt.get_cmap('Dark2')
# colorsF2a1 = iter(colormapF2a1(np.linspace(0,1,10)))


color_nopot = iter(
    cm.Reds(np.linspace(1 / 2, 1, len(k_a)))
)  # Go from 1/2 to 1 to display colours not too bright
color_gauss = iter(cm.Blues(np.linspace(1 / 2, 1, len(k_a))))
color_cauchy = iter(cm.Greens(np.linspace(1 / 2, 1, len(k_a))))

tau_ad_a = np.linspace(-1.35, -1.0, 100)
tau_ad_a = tau_a

plt.figure(figsize=(14, 7))

mylw = 1

for k in k_a:
    csq1d_nopot.set_k(k)
    csq1d_gauss.set_k(k)
    csq1d_cauchy.set_k(k)

    print(f"Searching for WKB scale in ({tau_min}, {tau_qt_c})")

    (Found_tau_i_nopot, tau_i_nopot) = csq1d_nopot.find_adiab_time_limit(
        Vexp, tau_min, -1.0e-2, 1.0e-9
    )
    (Found_tau_i_gauss, tau_i_gauss) = csq1d_gauss.find_adiab_time_limit(
        Vexp, tau_min, -1.0e-2, 1.0e-9
    )
    (Found_tau_i_cauchy, tau_i_cauchy) = csq1d_cauchy.find_adiab_time_limit(
        Vexp, tau_min, -1.0e-2, 1.0e-9
    )

    print(csq1d_nopot.eval_F1(Vexp, tau_i_nopot, k))
    print(csq1d_gauss.eval_F1(Vexp, tau_i_gauss, k))
    print(csq1d_cauchy.eval_F1(Vexp, tau_i_cauchy, k))

    print(csq1d_nopot.eval_adiab_at(Vexp, tau_i_nopot))
    print(csq1d_gauss.eval_adiab_at(Vexp, tau_i_gauss))
    print(csq1d_cauchy.eval_adiab_at(Vexp, tau_i_cauchy))

    F1_a_nopot = np.array([csq1d_nopot.eval_F1(Vexp, tau, k) for tau in tau_ad_a])
    F2_a_nopot = np.array([csq1d_nopot.eval_F2(Vexp, tau, k) for tau in tau_ad_a])
    nu_a_nopot = np.array([csq1d_nopot.eval_nu(Vexp, tau, k) for tau in tau_ad_a])

    F1_a_gauss = np.array([csq1d_gauss.eval_F1(Vexp, tau, k) for tau in tau_ad_a])
    F2_a_gauss = np.array([csq1d_gauss.eval_F2(Vexp, tau, k) for tau in tau_ad_a])
    nu_a_gauss = np.array([csq1d_gauss.eval_nu(Vexp, tau, k) for tau in tau_ad_a])

    F1_a_cauchy = np.array([csq1d_cauchy.eval_F1(Vexp, tau, k) for tau in tau_ad_a])
    F2_a_cauchy = np.array([csq1d_cauchy.eval_F2(Vexp, tau, k) for tau in tau_ad_a])
    nu_a_cauchy = np.array([csq1d_cauchy.eval_nu(Vexp, tau, k) for tau in tau_ad_a])

    csq1d_nopot.set_init_cond_adiab(Vexp, tau_i_nopot)
    csq1d_gauss.set_init_cond_adiab(Vexp, tau_i_gauss)
    csq1d_cauchy.set_init_cond_adiab(Vexp, tau_i_cauchy)

    csq1d_nopot.set_ti(tau_i_nopot)
    csq1d_nopot.set_tf(tau_f)
    csq1d_nopot.prepare(Vexp)

    csq1d_gauss.set_ti(tau_i_gauss)
    csq1d_gauss.set_tf(tau_f)
    csq1d_gauss.prepare(Vexp)

    csq1d_cauchy.set_ti(tau_i_cauchy)
    csq1d_cauchy.set_tf(tau_f)
    csq1d_cauchy.prepare(Vexp)

    tau_evol_a_nopot, tau_s_nopot = csq1d_nopot.get_time_array()
    tau_evol_a_nopot = np.array(tau_evol_a_nopot)

    tau_evol_a_gauss, tau_s_gauss = csq1d_gauss.get_time_array()
    tau_evol_a_gauss = np.array(tau_evol_a_gauss)

    tau_evol_a_cauchy, tau_s_cauchy = csq1d_cauchy.get_time_array()
    tau_evol_a_cauchy = np.array(tau_evol_a_cauchy)

    # PB_a_nopot = []
    # PE_a_nopot = []
    J11_a_nopot = []
    # J22_a_nopot = []

    # PB_a_gauss = []
    # PE_a_gauss = []
    J11_a_gauss = []
    # J22_a_gauss = []

    # PB_a_cauchy = []
    # PE_a_cauchy = []
    J11_a_cauchy = []
    # J22_a_cauchy = []

    for tau in tau_evol_a_nopot:
        PB, PE, J11, J22 = csq1d_nopot.eval_PB_PE(Vexp, tau)
        # PB_a_nopot.append(PB)
        # PE_a_nopot.append(PE)
        J11_a_nopot.append(J11)
        # J22_a_nopot.append(J22)

    for tau in tau_evol_a_gauss:
        PB, PE, J11, J22 = csq1d_gauss.eval_PB_PE(Vexp, tau)
        # PB_a_gauss.append(PB)
        # PE_a_gauss.append(PE)
        J11_a_gauss.append(J11)
        # J22_a_gauss.append(J22)

    for tau in tau_evol_a_cauchy:
        PB, PE, J11, J22 = csq1d_cauchy.eval_PB_PE(Vexp, tau)
        # PB_a_cauchy.append(PB)
        # PE_a_cauchy.append(PE)
        J11_a_cauchy.append(J11)
        # J22_a_cauchy.append(J22)

    # PB_a_nopot = np.array(PB_a_nopot)
    # PE_a_nopot = np.array(PE_a_nopot)
    J11_a_nopot = np.array(J11_a_nopot)
    # J22_a_nopot = np.array(J22_a_nopot)

    # PB_a_gauss = np.array(PB_a_gauss)
    # PE_a_gauss = np.array(PE_a_gauss)
    J11_a_gauss = np.array(J11_a_gauss)
    # J22_a_gauss = np.array(J22_a_gauss)

    # PB_a_cauchy = np.array(PB_a_cauchy)
    # PE_a_cauchy = np.array(PE_a_cauchy)
    J11_a_cauchy = np.array(J11_a_cauchy)
    # J22_a_cauchy = np.array(J22_a_cauchy)

    c_nopot = next(color_nopot)
    c_gauss = next(color_gauss)
    c_cauchy = next(color_cauchy)

    plt.plot(
        tau_evol_a_nopot,
        J11_a_nopot,
        c=c_nopot,
        lw=mylw,
        label=f"$J^{{nopot}}_{{11}}$ (k = $%s$)" % latex_float(k),
    )
    plt.plot(
        tau_evol_a_gauss,
        J11_a_gauss,
        c=c_gauss,
        lw=mylw,
        label=f"$J^{{gauss}}_{{11}}$ (k = $%s$)" % latex_float(k),
    )
    plt.plot(
        tau_evol_a_cauchy,
        J11_a_cauchy,
        c=c_cauchy,
        lw=mylw,
        label=f"$J^{{cauchy}}_{{11}}$ (k = $%s$)" % latex_float(k),
    )

leg = plt.legend(loc="center left", ncol=2, fontsize=16)
plt.yscale("log")
plt.xlabel(r"$\tau$")

plt.savefig(plotpath + "plotJ11_" + current_set + ".pdf")

plt.show()

In [ ]:
tau_f = 10.0

ki = 1.0
kf = 4.0e3
k_a = np.geomspace(ki, kf, 3)

B0 = 1.0e-50
beta0 = 8.0e-1
B1 = 1.0e50
beta1 = 8.0e-1

csq1d_nopot = PyCSQ1DMagCauchy(B0, beta0)
csq1d_gauss = PyCSQ1DMagGauss(B1, beta1)
csq1d_cauchy = PyCSQ1DMagCauchy(B1, beta1)

# Define map colours for easier visualisation
# Maps are defined at https://matplotlib.org/stable/tutorials/colors/colormaps.html
# colormapB0= plt.get_cmap('viridis')
# colorsB0 = iter(colormapB0(np.linspace(0,1,len(k_a))))

# colormapB1= plt.get_cmap('viridis')
# colorsB1 = iter(colormapB1(np.linspace(1,2,len(k_a))))

# colormapE0= plt.get_cmap('viridis')
# colorsE0 = iter(colormapE0(np.linspace(0,1,len(k_a))))

# colormapE1= plt.get_cmap('viridis')
# colorsE1 = iter(colormapE1(np.linspace(1,2,len(k_a))))

# colors = plt.cm.coolwarm(np.linspace(0, 1, len(k_a)))


color_nopot = iter(
    cm.Reds(np.linspace(1 / 2, 1, len(k_a)))
)  # Go from 1/2 to 1 to display colours not too bright
color_gauss = iter(cm.Blues(np.linspace(1 / 2, 1, len(k_a))))
color_cauchy = iter(cm.Greens(np.linspace(1 / 2, 1, len(k_a))))

tau_ad_a = np.linspace(-1.35, -1.0, 100)
tau_ad_a = tau_a

fig = plt.figure(figsize=(14, 7))

mylw = 1

for ik, k in enumerate(k_a):
    csq1d_nopot.set_k(k)
    csq1d_gauss.set_k(k)
    csq1d_cauchy.set_k(k)

    print(f"Searching for WKB scale in ({tau_min}, {tau_qt_c})")

    (Found_tau_i_nopot, tau_i_nopot) = csq1d_nopot.find_adiab_time_limit(
        Vexp, tau_min, -1.0e-2, 1.0e-9
    )
    (Found_tau_i_gauss, tau_i_gauss) = csq1d_gauss.find_adiab_time_limit(
        Vexp, tau_min, -1.0e-2, 1.0e-9
    )
    (Found_tau_i_cauchy, tau_i_cauchy) = csq1d_cauchy.find_adiab_time_limit(
        Vexp, tau_min, -1.0e-2, 1.0e-9
    )

    print(csq1d_nopot.eval_F1(Vexp, tau_i_nopot, k))
    print(csq1d_gauss.eval_F1(Vexp, tau_i_gauss, k))
    print(csq1d_cauchy.eval_F1(Vexp, tau_i_cauchy, k))

    print(csq1d_nopot.eval_adiab_at(Vexp, tau_i_nopot))
    print(csq1d_gauss.eval_adiab_at(Vexp, tau_i_gauss))
    print(csq1d_cauchy.eval_adiab_at(Vexp, tau_i_cauchy))

    F1_a_nopot = np.array([csq1d_nopot.eval_F1(Vexp, tau, k) for tau in tau_ad_a])
    F2_a_nopot = np.array([csq1d_nopot.eval_F2(Vexp, tau, k) for tau in tau_ad_a])
    nu_a_nopot = np.array([csq1d_nopot.eval_nu(Vexp, tau, k) for tau in tau_ad_a])

    F1_a_gauss = np.array([csq1d_gauss.eval_F1(Vexp, tau, k) for tau in tau_ad_a])
    F2_a_gauss = np.array([csq1d_gauss.eval_F2(Vexp, tau, k) for tau in tau_ad_a])
    nu_a_gauss = np.array([csq1d_gauss.eval_nu(Vexp, tau, k) for tau in tau_ad_a])

    F1_a_cauchy = np.array([csq1d_cauchy.eval_F1(Vexp, tau, k) for tau in tau_ad_a])
    F2_a_cauchy = np.array([csq1d_cauchy.eval_F2(Vexp, tau, k) for tau in tau_ad_a])
    nu_a_cauchy = np.array([csq1d_cauchy.eval_nu(Vexp, tau, k) for tau in tau_ad_a])

    csq1d_nopot.set_init_cond_adiab(Vexp, tau_i_nopot)
    csq1d_gauss.set_init_cond_adiab(Vexp, tau_i_gauss)
    csq1d_cauchy.set_init_cond_adiab(Vexp, tau_i_cauchy)

    csq1d_nopot.set_ti(tau_i_nopot)
    csq1d_nopot.set_tf(tau_f)
    csq1d_nopot.prepare(Vexp)

    csq1d_gauss.set_ti(tau_i_gauss)
    csq1d_gauss.set_tf(tau_f)
    csq1d_gauss.prepare(Vexp)

    csq1d_cauchy.set_ti(tau_i_cauchy)
    csq1d_cauchy.set_tf(tau_f)
    csq1d_cauchy.prepare(Vexp)

    tau_evol_a_nopot, tau_s_nopot = csq1d_nopot.get_time_array()
    tau_evol_a_nopot = np.array(tau_evol_a_nopot)

    tau_evol_a_gauss, tau_s_gauss = csq1d_gauss.get_time_array()
    tau_evol_a_gauss = np.array(tau_evol_a_gauss)

    tau_evol_a_cauchy, tau_s_cauchy = csq1d_cauchy.get_time_array()
    tau_evol_a_cauchy = np.array(tau_evol_a_cauchy)

    PB_a_nopot = []
    #     PE_a_nopot = []
    #     J11_a_nopot = []
    #     J22_a_nopot = []

    PB_a_gauss = []
    #     PE_a_gauss = []
    #     J11_a_gauss = []
    #     J22_a_gauss = []

    PB_a_cauchy = []
    #     PE_a_cauchy = []
    #     J11_a_cauchy = []
    #     J22_a_cauchy = []

    for tau in tau_evol_a_nopot:
        PB, PE, J11, J22 = csq1d_nopot.eval_PB_PE(Vexp, tau)
        PB_a_nopot.append(PB)
    #         PE_a_nopot.append(PE)
    #         J11_a_nopot.append(J11)
    #         J22_a_nopot.append(J22)

    for tau in tau_evol_a_gauss:
        PB, PE, J11, J22 = csq1d_gauss.eval_PB_PE(Vexp, tau)
        PB_a_gauss.append(PB)
    #         PE_a_gauss.append(PE)
    #         J11_a_gauss.append(J11)
    #         J22_a_gauss.append(J22)

    for tau in tau_evol_a_cauchy:
        PB, PE, J11, J22 = csq1d_cauchy.eval_PB_PE(Vexp, tau)
        PB_a_cauchy.append(PB)
    #         PE_a_cauchy.append(PE)
    #         J11_a_cauchy.append(J11)
    #         J22_a_cauchy.append(J22)

    PB_a_nopot = np.array(PB_a_nopot)
    #     PE_a_nopot = np.array(PE_a_nopot)
    #     J11_a_nopot = np.array(J11_a_nopot)
    #     J22_a_nopot = np.array(J22_a_nopot)

    PB_a_gauss = np.array(PB_a_gauss)
    #     PE_a_gauss = np.array(PE_a_gauss)
    #     J11_a_gauss = np.array(J11_a_gauss)
    #     J22_a_gauss = np.array(J22_a_gauss)

    PB_a_cauchy = np.array(PB_a_cauchy)
    #     PE_a_cauchy = np.array(PE_a_cauchy)
    #     J11_a_cauchy = np.array(J11_a_cauchy)
    #     J22_a_cauchy = np.array(J22_a_cauchy)

    c_nopot = next(color_nopot)
    c_gauss = next(color_gauss)
    c_cauchy = next(color_cauchy)

    plt.plot(
        tau_evol_a_nopot,
        PB_a_nopot,
        c=c_nopot,
        lw=mylw,
        label=f"$P_{{B^{{nopot}}}}$ (k = $%s$)" % latex_float(k),
    )
    plt.plot(
        tau_evol_a_gauss,
        PB_a_gauss,
        c=c_gauss,
        lw=mylw,
        label=f"$P_{{B^{{gauss}}}}$ (k = $%s$)" % latex_float(k),
    )
    plt.plot(
        tau_evol_a_cauchy,
        PB_a_cauchy,
        c=c_cauchy,
        lw=mylw,
        label=f"$P_{{B^{{cauchy}}}}$ (k = $%s$)" % latex_float(k),
    )

#     plt.plot (tau_evol_a_nopot, PE_a_nopot, c=c_nopot, lw=mylw, label = f'$P_{{E^{{nopot}}}}$ (k = $%s$)'% latex_float (k))
#     plt.plot (tau_evol_a_gauss, PE_a_gauss, c=c_gauss, lw=mylw, label = f'$P^{{E^{{gauss}}}}$ (k = $%s$)'% latex_float (k))
#     plt.plot (tau_evol_a_cauchy, PE_a_cauchy, c=c_cauchy, lw=mylw, label = f'$P^{{E^{{cauchy}}}}$ (k = $%s$)'% latex_float (k))


plt.xlabel(r"$\tau$")
plt.yscale("log")
leg = plt.legend(loc="best", ncol=2, fontsize=16)

plt.savefig(plotpath + "plotPB_" + current_set + ".pdf")

plt.show()

In [ ]:
tau_f = 10.0

ki = 1.0
kf = 4.0e3
k_a = np.geomspace(ki, kf, 3)

B0 = 1.0e-50
beta0 = 8.0e-1
B1 = 1.0e50
beta1 = 8.0e-1

csq1d_nopot = PyCSQ1DMagCauchy(B0, beta0)
csq1d_gauss = PyCSQ1DMagGauss(B1, beta1)
csq1d_cauchy = PyCSQ1DMagCauchy(B1, beta1)

# Define map colours for easier visualisation
# Maps are defined at https://matplotlib.org/stable/tutorials/colors/colormaps.html
# colormapB0= plt.get_cmap('viridis')
# colorsB0 = iter(colormapB0(np.linspace(0,1,len(k_a))))

# colormapB1= plt.get_cmap('viridis')
# colorsB1 = iter(colormapB1(np.linspace(1,2,len(k_a))))

# colormapE0= plt.get_cmap('viridis')
# colorsE0 = iter(colormapE0(np.linspace(0,1,len(k_a))))

# colormapE1= plt.get_cmap('viridis')
# colorsE1 = iter(colormapE1(np.linspace(1,2,len(k_a))))

# colors = plt.cm.coolwarm(np.linspace(0, 1, len(k_a)))


color_nopot = iter(
    cm.Reds(np.linspace(1 / 2, 1, len(k_a)))
)  # Go from 1/2 to 1 to display colours not too bright
color_gauss = iter(cm.Blues(np.linspace(1 / 2, 1, len(k_a))))
color_cauchy = iter(cm.Greens(np.linspace(1 / 2, 1, len(k_a))))

tau_ad_a = np.linspace(-1.35, -1.0, 100)
tau_ad_a = tau_a

fig = plt.figure(figsize=(14, 7))

mylw = 1

for ik, k in enumerate(k_a):
    csq1d_nopot.set_k(k)
    csq1d_gauss.set_k(k)
    csq1d_cauchy.set_k(k)

    print(f"Searching for WKB scale in ({tau_min}, {tau_qt_c})")

    (Found_tau_i_nopot, tau_i_nopot) = csq1d_nopot.find_adiab_time_limit(
        Vexp, tau_min, -1.0e-2, 1.0e-9
    )
    (Found_tau_i_gauss, tau_i_gauss) = csq1d_gauss.find_adiab_time_limit(
        Vexp, tau_min, -1.0e-2, 1.0e-9
    )
    (Found_tau_i_cauchy, tau_i_cauchy) = csq1d_cauchy.find_adiab_time_limit(
        Vexp, tau_min, -1.0e-2, 1.0e-9
    )

    print(csq1d_nopot.eval_F1(Vexp, tau_i_nopot, k))
    print(csq1d_gauss.eval_F1(Vexp, tau_i_gauss, k))
    print(csq1d_cauchy.eval_F1(Vexp, tau_i_cauchy, k))

    print(csq1d_nopot.eval_adiab_at(Vexp, tau_i_nopot))
    print(csq1d_gauss.eval_adiab_at(Vexp, tau_i_gauss))
    print(csq1d_cauchy.eval_adiab_at(Vexp, tau_i_cauchy))

    F1_a_nopot = np.array([csq1d_nopot.eval_F1(Vexp, tau, k) for tau in tau_ad_a])
    F2_a_nopot = np.array([csq1d_nopot.eval_F2(Vexp, tau, k) for tau in tau_ad_a])
    nu_a_nopot = np.array([csq1d_nopot.eval_nu(Vexp, tau, k) for tau in tau_ad_a])

    F1_a_gauss = np.array([csq1d_gauss.eval_F1(Vexp, tau, k) for tau in tau_ad_a])
    F2_a_gauss = np.array([csq1d_gauss.eval_F2(Vexp, tau, k) for tau in tau_ad_a])
    nu_a_gauss = np.array([csq1d_gauss.eval_nu(Vexp, tau, k) for tau in tau_ad_a])

    F1_a_cauchy = np.array([csq1d_cauchy.eval_F1(Vexp, tau, k) for tau in tau_ad_a])
    F2_a_cauchy = np.array([csq1d_cauchy.eval_F2(Vexp, tau, k) for tau in tau_ad_a])
    nu_a_cauchy = np.array([csq1d_cauchy.eval_nu(Vexp, tau, k) for tau in tau_ad_a])

    csq1d_nopot.set_init_cond_adiab(Vexp, tau_i_nopot)
    csq1d_gauss.set_init_cond_adiab(Vexp, tau_i_gauss)
    csq1d_cauchy.set_init_cond_adiab(Vexp, tau_i_cauchy)

    csq1d_nopot.set_ti(tau_i_nopot)
    csq1d_nopot.set_tf(tau_f)
    csq1d_nopot.prepare(Vexp)

    csq1d_gauss.set_ti(tau_i_gauss)
    csq1d_gauss.set_tf(tau_f)
    csq1d_gauss.prepare(Vexp)

    csq1d_cauchy.set_ti(tau_i_cauchy)
    csq1d_cauchy.set_tf(tau_f)
    csq1d_cauchy.prepare(Vexp)

    tau_evol_a_nopot, tau_s_nopot = csq1d_nopot.get_time_array()
    tau_evol_a_nopot = np.array(tau_evol_a_nopot)

    tau_evol_a_gauss, tau_s_gauss = csq1d_gauss.get_time_array()
    tau_evol_a_gauss = np.array(tau_evol_a_gauss)

    tau_evol_a_cauchy, tau_s_cauchy = csq1d_cauchy.get_time_array()
    tau_evol_a_cauchy = np.array(tau_evol_a_cauchy)

    #     PB_a_nopot = []
    PE_a_nopot = []
    #     J11_a_nopot = []
    #     J22_a_nopot = []

    #     PB_a_gauss = []
    PE_a_gauss = []
    #     J11_a_gauss = []
    #     J22_a_gauss = []

    #     PB_a_cauchy = []
    PE_a_cauchy = []
    #     J11_a_cauchy = []
    #     J22_a_cauchy = []

    for tau in tau_evol_a_nopot:
        PB, PE, J11, J22 = csq1d_nopot.eval_PB_PE(Vexp, tau)
        #         PB_a_nopot.append(PB)
        PE_a_nopot.append(PE)
    #         J11_a_nopot.append(J11)
    #         J22_a_nopot.append(J22)

    for tau in tau_evol_a_gauss:
        PB, PE, J11, J22 = csq1d_gauss.eval_PB_PE(Vexp, tau)
        #         PB_a_gauss.append(PB)
        PE_a_gauss.append(PE)
    #         J11_a_gauss.append(J11)
    #         J22_a_gauss.append(J22)

    for tau in tau_evol_a_cauchy:
        PB, PE, J11, J22 = csq1d_cauchy.eval_PB_PE(Vexp, tau)
        #         PB_a_cauchy.append(PB)
        PE_a_cauchy.append(PE)
    #         J11_a_cauchy.append(J11)
    #         J22_a_cauchy.append(J22)

    #     PB_a_nopot = np.array(PB_a_nopot)
    PE_a_nopot = np.array(PE_a_nopot)
    #     J11_a_nopot = np.array(J11_a_nopot)
    #     J22_a_nopot = np.array(J22_a_nopot)

    #     PB_a_gauss = np.array(PB_a_gauss)
    PE_a_gauss = np.array(PE_a_gauss)
    #     J11_a_gauss = np.array(J11_a_gauss)
    #     J22_a_gauss = np.array(J22_a_gauss)

    #     PB_a_cauchy = np.array(PB_a_cauchy)
    PE_a_cauchy = np.array(PE_a_cauchy)
    #     J11_a_cauchy = np.array(J11_a_cauchy)
    #     J22_a_cauchy = np.array(J22_a_cauchy)

    c_nopot = next(color_nopot)
    c_gauss = next(color_gauss)
    c_cauchy = next(color_cauchy)

    #     plt.plot (tau_evol_a_nopot, PB_a_nopot, c=c_nopot, lw=mylw, label = f'$P_{{B^{{nopot}}}}$ (k = $%s$)'% latex_float (k))
    #     plt.plot (tau_evol_a_gauss, PB_a_gauss, c=c_gauss, lw=mylw, label = f'$P^{{B^{{gauss}}}}$ (k = $%s$)'% latex_float (k))
    #     plt.plot (tau_evol_a_cauchy, PB_a_cauchy, c=c_cauchy, lw=mylw, label = f'$P^{{B^{{cauchy}}}}$ (k = $%s$)'% latex_float (k))

    plt.plot(
        tau_evol_a_nopot,
        PE_a_nopot,
        c=c_nopot,
        lw=mylw,
        label=f"$P_{{E^{{nopot}}}}$ (k = $%s$)" % latex_float(k),
    )
    plt.plot(
        tau_evol_a_gauss,
        PE_a_gauss,
        c=c_gauss,
        lw=mylw,
        label=f"$P_{{E^{{gauss}}}}$ (k = $%s$)" % latex_float(k),
    )
    plt.plot(
        tau_evol_a_cauchy,
        PE_a_cauchy,
        c=c_cauchy,
        lw=mylw,
        label=f"$P_{{E^{{cauchy}}}}$ (k = $%s$)" % latex_float(k),
    )


plt.xlabel(r"$\tau$")
plt.yscale("log")
leg = plt.legend(loc="best", ncol=2, fontsize=16)
plt.savefig(plotpath + "plotPE_" + current_set + ".pdf")

plt.show()

In [ ]:
tau_f = 10.0

ki = 1.0
kf = 4.0e3
k_a = np.geomspace(ki, kf, 3)

B1 = 1.0e10
beta1 = 8.0e-1

B2 = 1.0e20
beta2 = 8.0e-1

csq1d2 = PyCSQ1DMagScalarField(B2, beta2)
csq1d1 = PyCSQ1DMagScalarField(B1, beta1)

# Define map colours for easier visualisation
# Maps are defined at https://matplotlib.org/stable/tutorials/colors/colormaps.html
# colormapB0= plt.get_cmap('viridis')
# colorsB0 = iter(colormapB0(np.linspace(0,1,len(k_a))))

# colormapB1= plt.get_cmap('viridis')
# colorsB1 = iter(colormapB1(np.linspace(1,2,len(k_a))))

# colormapE0= plt.get_cmap('viridis')
# colorsE0 = iter(colormapE0(np.linspace(0,1,len(k_a))))

# colormapE1= plt.get_cmap('viridis')
# colorsE1 = iter(colormapE1(np.linspace(1,2,len(k_a))))

colors = plt.cm.coolwarm(np.linspace(0, 1, len(k_a)))


tau_ad_a = np.linspace(-1.35, -1.0, 100)
tau_ad_a = tau_a

fig = plt.figure(figsize=(14, 7))

mylw = 1

for ik, k in enumerate(k_a[:2]):
    csq1d2.set_k(k)
    csq1d1.set_k(k)

    print(f"Searching for WKB scale in ({tau_min}, {tau_qt_c})")

    (Found_tau_i2, tau_i2) = csq1d2.find_adiab_time_limit(
        Vexp, tau_min, -1.0e-2, 1.0e-9
    )
    (Found_tau_i1, tau_i1) = csq1d1.find_adiab_time_limit(
        Vexp, tau_min, -1.0e-2, 1.0e-9
    )

    print(csq1d2.eval_F1(Vexp, tau_i2, k))
    print(csq1d1.eval_F1(Vexp, tau_i1, k))

    print(csq1d2.eval_adiab_at(Vexp, tau_i2))
    print(csq1d1.eval_adiab_at(Vexp, tau_i1))

    F1_a2 = np.array([csq1d2.eval_F1(Vexp, tau, k) for tau in tau_ad_a])
    F2_a2 = np.array([csq1d2.eval_F2(Vexp, tau, k) for tau in tau_ad_a])
    nu_a2 = np.array([csq1d2.eval_nu(Vexp, tau, k) for tau in tau_ad_a])

    F1_a1 = np.array([csq1d1.eval_F1(Vexp, tau, k) for tau in tau_ad_a])
    F2_a1 = np.array([csq1d1.eval_F2(Vexp, tau, k) for tau in tau_ad_a])
    nu_a1 = np.array([csq1d1.eval_nu(Vexp, tau, k) for tau in tau_ad_a])

    csq1d2.set_init_cond_adiab(Vexp, tau_i2)
    csq1d1.set_init_cond_adiab(Vexp, tau_i1)

    csq1d2.set_ti(tau_i2)
    csq1d2.set_tf(tau_f)
    csq1d2.prepare(Vexp)

    csq1d1.set_ti(tau_i1)
    csq1d1.set_tf(tau_f)
    csq1d1.prepare(Vexp)

    tau_evol_a2, tau_s2 = csq1d2.get_time_array()
    tau_evol_a2 = np.array(tau_evol_a2)

    tau_evol_a1, tau_s1 = csq1d1.get_time_array()
    tau_evol_a1 = np.array(tau_evol_a1)

    PB_a2 = []
    PE_a2 = []
    J11_a2 = []
    J22_a2 = []

    PB_a1 = []
    PE_a1 = []
    J11_a1 = []
    J22_a1 = []

    for tau in tau_evol_a2:
        PB, PE, J11, J22 = csq1d2.eval_PB_PE(Vexp, tau)
        PB_a2.append(PB)
        PE_a2.append(PE)
        J11_a2.append(J11)
        J22_a2.append(J22)

    for tau in tau_evol_a1:
        PB, PE, J11, J22 = csq1d1.eval_PB_PE(Vexp, tau)
        PB_a1.append(PB)
        PE_a1.append(PE)
        J11_a1.append(J11)
        J22_a1.append(J22)

    PB_a2 = np.array(PB_a2)
    PE_a2 = np.array(PE_a2)
    J11_a2 = np.array(J11_a2)
    J22_a2 = np.array(J22_a2)

    PB_a1 = np.array(PB_a1)
    PE_a1 = np.array(PE_a1)
    J11_a1 = np.array(J11_a1)
    J22_a1 = np.array(J22_a1)

    plt.plot(
        tau_evol_a2,
        PB_a2,
        lw=mylw,
        label=f"$P_{{B^2}}(k_s = {int(k)})$",
        color=colors[ik],
    )
    plt.plot(
        tau_evol_a1,
        PB_a1,
        lw=mylw,
        linestyle="dashed",
        label=f"$P_{{B^1}}(k_s = {int(k)})$",
        color=colors[ik],
    )

    # plt.plot (tau_evol_a2, PE_a2, lw=mylw, linestyle='dotted', label = f'$P_{{E^2}}(k_s = {int(k)})$',color=colors[ik])
    # plt.plot (tau_evol_a1, PE_a1, lw=mylw, linestyle='dashdot', label = f'$P_{{E^1}}(k_s = {int(k)})$',color=colors[ik])


#     plt.plot (tau_evol_a0, PB_a0, lw=mylw, label = f'$P_{{B^0}}(k_s = {int(k)})$',color=next(colorsB0))
#     plt.plot (tau_evol_a1, PB_a1, lw=mylw, label = f'$P_{{B^1}}(k_s = {int(k)})$',color=next(colorsB1))

#     plt.plot (tau_evol_a0, PE_a0, lw=mylw, linestyle='--', label = f'$P_{{E^0}}(k_s = {int(k)})$',color=next(colorsE0))
#     plt.plot (tau_evol_a1, PE_a1, lw=mylw, linestyle='--', label = f'$P_{{E^1}}(k_s = {int(k)})$',color=next(colorsE1))

leg = plt.legend(loc="best", ncol=2, fontsize=13)
plt.grid()
plt.yscale("log")
plt.xlim(-5.0, 5.0)
plt.ylim(1.0e-38, None)
plt.show()
fig.savefig("PE_PB_testbeta2.pdf")

In [ ]:
### Magnetic power spectrum

tau_f = 10.0

ki = 1.0
kf = 4.0e3
k_a = np.geomspace(ki, kf, 2)

B0 = 1.0e-50
beta0 = 8.0e-1
B1 = 1.0e50
beta1 = 8.0e3

csq1d_nopot = PyCSQ1DMagCauchy(B0, beta0)
csq1d_gauss = PyCSQ1DMagGauss(B1, beta1)
csq1d_cauchy = PyCSQ1DMagCauchy(B1, beta1)

color_nopot = iter(
    cm.Reds(np.linspace(1 / 2, 1, len(k_a)))
)  # Go from 1/2 to 1 to display colours not too bright
color_gauss = iter(cm.Blues(np.linspace(1 / 2, 1, len(k_a))))
color_cauchy = iter(cm.Greens(np.linspace(1 / 2, 1, len(k_a))))

tau_ad_a = np.linspace(-1.35, -1.0, 100)
tau_ad_a = tau_a

fig = plt.figure(figsize=(14, 7))
ax = fig.add_subplot(111)

mylw = 1

PB_a_nopot = []
#     PE_a_nopot = []
#     J11_a_nopot = []
#     J22_a_nopot = []

PB_a_gauss = []
#     PE_a_gauss = []
#     J11_a_gauss = []
#     J22_a_gauss = []

PB_a_cauchy = []
#     PE_a_cauchy = []
#     J11_a_cauchy = []
#     J22_a_cauchy = []

for ik, k in enumerate(k_a):
    csq1d_nopot.set_k(k)
    csq1d_gauss.set_k(k)
    csq1d_cauchy.set_k(k)

    print(f"Searching for WKB scale in ({tau_min}, {tau_qt_c})")

    (Found_tau_i_nopot, tau_i_nopot) = csq1d_nopot.find_adiab_time_limit(
        Vexp, tau_min, -1.0e-2, 1.0e-9
    )
    (Found_tau_i_gauss, tau_i_gauss) = csq1d_gauss.find_adiab_time_limit(
        Vexp, tau_min, -1.0e-2, 1.0e-9
    )
    (Found_tau_i_cauchy, tau_i_cauchy) = csq1d_cauchy.find_adiab_time_limit(
        Vexp, tau_min, -1.0e-2, 1.0e-9
    )

    print(csq1d_nopot.eval_F1(Vexp, tau_i_nopot, k))
    print(csq1d_gauss.eval_F1(Vexp, tau_i_gauss, k))
    print(csq1d_cauchy.eval_F1(Vexp, tau_i_cauchy, k))

    print(csq1d_nopot.eval_adiab_at(Vexp, tau_i_nopot))
    print(csq1d_gauss.eval_adiab_at(Vexp, tau_i_gauss))
    print(csq1d_cauchy.eval_adiab_at(Vexp, tau_i_cauchy))

    F1_a_nopot = np.array([csq1d_nopot.eval_F1(Vexp, tau, k) for tau in tau_ad_a])
    F2_a_nopot = np.array([csq1d_nopot.eval_F2(Vexp, tau, k) for tau in tau_ad_a])
    nu_a_nopot = np.array([csq1d_nopot.eval_nu(Vexp, tau, k) for tau in tau_ad_a])

    F1_a_gauss = np.array([csq1d_gauss.eval_F1(Vexp, tau, k) for tau in tau_ad_a])
    F2_a_gauss = np.array([csq1d_gauss.eval_F2(Vexp, tau, k) for tau in tau_ad_a])
    nu_a_gauss = np.array([csq1d_gauss.eval_nu(Vexp, tau, k) for tau in tau_ad_a])

    F1_a_cauchy = np.array([csq1d_cauchy.eval_F1(Vexp, tau, k) for tau in tau_ad_a])
    F2_a_cauchy = np.array([csq1d_cauchy.eval_F2(Vexp, tau, k) for tau in tau_ad_a])
    nu_a_cauchy = np.array([csq1d_cauchy.eval_nu(Vexp, tau, k) for tau in tau_ad_a])

    csq1d_nopot.set_init_cond_adiab(Vexp, tau_i_nopot)
    csq1d_gauss.set_init_cond_adiab(Vexp, tau_i_gauss)
    csq1d_cauchy.set_init_cond_adiab(Vexp, tau_i_cauchy)

    tau_f = Vexp.tau_xe(1.0)
    csq1d_nopot.set_ti(tau_i_nopot)
    csq1d_nopot.set_tf(tau_f)
    csq1d_nopot.prepare(Vexp)

    csq1d_gauss.set_ti(tau_i_gauss)
    csq1d_gauss.set_tf(tau_f)
    csq1d_gauss.prepare(Vexp)

    csq1d_cauchy.set_ti(tau_i_cauchy)
    csq1d_cauchy.set_tf(tau_f)
    csq1d_cauchy.prepare(Vexp)

    tau_evol_a_nopot, tau_s_nopot = csq1d_nopot.get_time_array()
    tau_evol_a_nopot = np.array(tau_evol_a_nopot)

    tau_evol_a_gauss, tau_s_gauss = csq1d_gauss.get_time_array()
    tau_evol_a_gauss = np.array(tau_evol_a_gauss)

    tau_evol_a_cauchy, tau_s_cauchy = csq1d_cauchy.get_time_array()
    tau_evol_a_cauchy = np.array(tau_evol_a_cauchy)

    PB, PE, J11, J22 = csq1d_nopot.eval_PB_PE(Vexp, Vexp.tau_xe(1.0))
    PB_a_nopot.append(PB)
    #         PE_a_nopot.append(PE)

    PB, PE, J11, J22 = csq1d_gauss.eval_PB_PE(Vexp, Vexp.tau_xe(1.0))
    PB_a_gauss.append(PB)
    #         PE_a_gauss.append(PE)

    PB, PE, J11, J22 = csq1d_cauchy.eval_PB_PE(Vexp, Vexp.tau_xe(1.0))
    PB_a_cauchy.append(PB)
#         PE_a_cauchy.append(PE)


PB_a_nopot = np.array(PB_a_nopot)
#     PE_a_nopot = np.array(PE_a_nopot)

PB_a_gauss = np.array(PB_a_gauss)
#     PE_a_gauss = np.array(PE_a_gauss)

PB_a_cauchy = np.array(PB_a_cauchy)

plt.plot(
    k_a,
    np.sqrt(PB_a_nopot * csq1d_nopot.RH**6) * 1.0e9,
    lw=mylw,
    label=f"$B^{{nopot}}$",
)
plt.plot(
    k_a,
    np.sqrt(PB_a_gauss * csq1d_gauss.RH**6) * 1.0e9,
    lw=mylw,
    label=f"$B^{{gauss}}$",
)
plt.plot(
    k_a,
    np.sqrt(PB_a_cauchy * csq1d_cauchy.RH**6) * 1.0e9,
    lw=mylw,
    label=f"$B^{{cauchy}}$",
)

#     plt.plot (tau_evol_a_nopot, PE_a_nopot, c=c_nopot, lw=mylw, label = f'$P_{{E^{{nopot}}}}$ (k = $%s$)'% latex_float (k))
#     plt.plot (tau_evol_a_gauss, PE_a_gauss, c=c_gauss, lw=mylw, label = f'$P^{{E^{{gauss}}}}$ (k = $%s$)'% latex_float (k))
#     plt.plot (tau_evol_a_cauchy, PE_a_cauchy, c=c_cauchy, lw=mylw, label = f'$P^{{E^{{cauchy}}}}$ (k = $%s$)'% latex_float (k))


# Spectral indices: magnetic = nB, electric = nE
# nE = math.log (PE_a_nopot[-1] / PE_a_nopot[0]) / math.log (k_a[-1] / k_a[0])
nB_nopot = math.log(PB_a_nopot[-1] / PB_a_nopot[0]) / math.log(k_a[-1] / k_a[0])
print("Magnetic spectral index no_pot: ", nB_nopot)
nB_gauss = math.log(PB_a_gauss[-1] / PB_a_gauss[0]) / math.log(k_a[-1] / k_a[0])
print("Magnetic spectral index gauss: ", nB_gauss)
nB_cauchy = math.log(PB_a_cauchy[-1] / PB_a_cauchy[0]) / math.log(k_a[-1] / k_a[0])
print("Magnetic spectral index cauchy: ", nB_cauchy)

plt.xlabel("$k R_H$", fontsize=16)
plt.ylabel("$B_{\lambda} (nG)$", fontsize=16)
plt.yscale("log")
plt.xscale("log")
plt.grid(b=True, which="both", linestyle=":", color="0.5", linewidth=0.5)
leg = plt.legend(loc="best", fontsize=16)


# Top x-axis
def forward(x):
    return [csq1d_nopot.RH / xi if xi > 0.0 else 1.0e30 for xi in x]


def inverse(x):
    return [csq1d_nopot.RH / xi if xi > 0.0 else 1.0e30 for xi in x]


ax2 = ax.secondary_xaxis("top", functions=(forward, inverse))
ax2.set_xlabel("$\lambda (Mpc)$")

# Observational limits

# print(ax.get_ylim(),math.log10(ax.get_ylim()[1]),math.log10(ax.get_ylim()[0]))
y_center = 10 ** (
    (math.log10(ax.get_ylim()[1]) + math.log10(ax.get_ylim()[0])) / 2.0
)  # Get center of y-axis
plt.axvline(1.0, color="green")  # Draw vertical line for Hubble radius
plt.text(
    1.05,
    y_center,
    "Hubble radius",
    color="green",
    rotation=90,
    verticalalignment="center",
)  # Add text

# print(np.sqrt(PB_a_nopot[-1]*csq1d_nopot.RH**6)*1.0e9,np.sqrt(PB_a_gauss[-1]*csq1d_gauss.RH**6)*1.0e9,np.sqrt(PB_a_cauchy[-1]*csq1d_cauchy.RH**6)*1.0e9)

# Check if y range reaches observational bounds before drawing the thresholds
# First check is for CMB, second for dynamo. If CMB is reached, it also draws the dynamo threshold
if (
    (np.sqrt(PB_a_nopot[-1] * csq1d_nopot.RH**6) * 1.0e9 > 0.9)
    or (np.sqrt(PB_a_gauss[-1] * csq1d_gauss.RH**6) * 1.0e9 > 0.9)
    or (np.sqrt(PB_a_cauchy[-1] * csq1d_cauchy.RH**6) * 1.0e9 > 0.9)
):
    x_center = 10 ** (
        (math.log10(ax.get_xlim()[1]) + math.log10(ax.get_xlim()[0])) / 2.0
    )  # Get center of x-axis

    plt.axhline(1.0, color="red")  # Draw horizontal line for CMB threshold
    plt.text(
        x_center,
        5.0e-3,
        "CMB limit $(1 nG)$",
        color="red",
        horizontalalignment="center",
    )  # Add text

    plt.axhline(1.0e-12, color="blue")  # Draw horizontal line for dynamo threshold
    plt.text(
        x_center,
        5.0e-12,
        "Dynamo threshold $(10^{-12} nG)$",
        color="blue",
        horizontalalignment="center",
    )  # Add text
elif (
    (np.sqrt(PB_a_nopot[-1] * csq1d_nopot.RH**6) * 1.0e9 > 0.9e-13)
    or (np.sqrt(PB_a_gauss[-1] * csq1d_gauss.RH**6) * 1.0e9 > 0.9e-13)
    or (np.sqrt(PB_a_cauchy[-1] * csq1d_cauchy.RH**6) * 1.0e9 > 0.9e-13)
):
    x_center = 10 ** (
        (math.log10(ax.get_xlim()[1]) + math.log10(ax.get_xlim()[0])) / 2.0
    )  # Get center of x-axis

    plt.axhline(1.0e-12, color="blue")  # Draw horizontal line for dynamo threshold
    plt.text(
        x_center,
        2.0e-12,
        "Dynamo threshold",
        color="blue",
        horizontalalignment="center",
    )  # Add text

plt.savefig(
    plotpath + "plotB_" + current_set + "_" + str(B1) + "_" + str(beta1) + ".pdf"
)

plt.show()

In [ ]:
### Backreaction


tau_f = 10.0

ki = 1.0
# kf = ki
kf = 4.0e3
k_a = np.geomspace(ki, kf, 3)

B0 = 1.0e-50
beta0 = 8.0e-1
B1 = 1.0e50
beta1 = 8.0e-1


csq1d_nopot = PyCSQ1DMagCauchy(B0, beta0)
csq1d_gauss = PyCSQ1DMagGauss(B1, beta1)
csq1d_cauchy = PyCSQ1DMagCauchy(B1, beta1)

color_nopot = iter(
    cm.Reds(np.linspace(1 / 2, 1, len(k_a)))
)  # Go from 1/2 to 1 to display colours not too bright
color_gauss = iter(cm.Blues(np.linspace(1 / 2, 1, len(k_a))))
color_cauchy = iter(cm.Greens(np.linspace(1 / 2, 1, len(k_a))))

tau_ad_a = np.linspace(-1.35, -1.0, 100)
tau_ad_a = tau_a

fig = plt.figure(figsize=(14, 7))


# EM field energy density

mylw = 1

for ik, k in enumerate(k_a):
    csq1d_nopot.set_k(k)
    csq1d_gauss.set_k(k)
    csq1d_cauchy.set_k(k)

    print(f"Searching for WKB scale in ({tau_min}, {tau_qt_c})")

    (Found_tau_i_nopot, tau_i_nopot) = csq1d_nopot.find_adiab_time_limit(
        Vexp, tau_min, -1.0e-2, 1.0e-9
    )
    (Found_tau_i_gauss, tau_i_gauss) = csq1d_gauss.find_adiab_time_limit(
        Vexp, tau_min, -1.0e-2, 1.0e-9
    )
    (Found_tau_i_cauchy, tau_i_cauchy) = csq1d_cauchy.find_adiab_time_limit(
        Vexp, tau_min, -1.0e-2, 1.0e-9
    )

    print(csq1d_nopot.eval_F1(Vexp, tau_i_nopot, k))
    print(csq1d_gauss.eval_F1(Vexp, tau_i_gauss, k))
    print(csq1d_cauchy.eval_F1(Vexp, tau_i_cauchy, k))

    print(csq1d_nopot.eval_adiab_at(Vexp, tau_i_nopot))
    print(csq1d_gauss.eval_adiab_at(Vexp, tau_i_gauss))
    print(csq1d_cauchy.eval_adiab_at(Vexp, tau_i_cauchy))

    F1_a_nopot = np.array([csq1d_nopot.eval_F1(Vexp, tau, k) for tau in tau_ad_a])
    F2_a_nopot = np.array([csq1d_nopot.eval_F2(Vexp, tau, k) for tau in tau_ad_a])
    nu_a_nopot = np.array([csq1d_nopot.eval_nu(Vexp, tau, k) for tau in tau_ad_a])

    F1_a_gauss = np.array([csq1d_gauss.eval_F1(Vexp, tau, k) for tau in tau_ad_a])
    F2_a_gauss = np.array([csq1d_gauss.eval_F2(Vexp, tau, k) for tau in tau_ad_a])
    nu_a_gauss = np.array([csq1d_gauss.eval_nu(Vexp, tau, k) for tau in tau_ad_a])

    F1_a_cauchy = np.array([csq1d_cauchy.eval_F1(Vexp, tau, k) for tau in tau_ad_a])
    F2_a_cauchy = np.array([csq1d_cauchy.eval_F2(Vexp, tau, k) for tau in tau_ad_a])
    nu_a_cauchy = np.array([csq1d_cauchy.eval_nu(Vexp, tau, k) for tau in tau_ad_a])

    csq1d_nopot.set_init_cond_adiab(Vexp, tau_i_nopot)
    csq1d_gauss.set_init_cond_adiab(Vexp, tau_i_gauss)
    csq1d_cauchy.set_init_cond_adiab(Vexp, tau_i_cauchy)

    csq1d_nopot.set_ti(tau_i_nopot)
    csq1d_nopot.set_tf(tau_f)
    csq1d_nopot.prepare(Vexp)

    csq1d_gauss.set_ti(tau_i_gauss)
    csq1d_gauss.set_tf(tau_f)
    csq1d_gauss.prepare(Vexp)

    csq1d_cauchy.set_ti(tau_i_cauchy)
    csq1d_cauchy.set_tf(tau_f)
    csq1d_cauchy.prepare(Vexp)

    tau_evol_a_nopot, tau_s_nopot = csq1d_nopot.get_time_array()
    tau_evol_a_nopot = np.array(tau_evol_a_nopot)

    tau_evol_a_gauss, tau_s_gauss = csq1d_gauss.get_time_array()
    tau_evol_a_gauss = np.array(tau_evol_a_gauss)

    tau_evol_a_cauchy, tau_s_cauchy = csq1d_cauchy.get_time_array()
    tau_evol_a_cauchy = np.array(tau_evol_a_cauchy)

    rho_phi_a = []

    PB_a_nopot = []
    PE_a_nopot = []
    sf_a_nopot = []

    PB_a_gauss = []
    PE_a_gauss = []
    sf_a_gauss = []

    PB_a_cauchy = []
    PE_a_cauchy = []
    sf_a_cauchy = []

    for tau in tau_evol_a_nopot:
        # Scale factor in terms of tau
        alpha_a_nopot = Vexp.alpha(tau)
        sf_a_nopot.append(np.exp(alpha_a_nopot) / Vexp.props.xb)
        # Scalar field energy density. x and y are defined as (x,y) -> (kappa/sqrt(6))*(x,y)
        # rho_phi = H^2 (x^2+y^2) / 2

        rho_phi_a.append(
            (
                (Vexp.x_y(tau)[0] ** 2 + Vexp.x_y(tau)[1] ** 2)
                / 2.0
                / (csq1d_nopot.RH * csq1d_nopot.RH)
            )
        )

        PB, PE, J11, J22 = csq1d_nopot.eval_PB_PE(Vexp, tau)
        PB_a_nopot.append(PB)
        PE_a_nopot.append(PE)

    for tau in tau_evol_a_gauss:
        # Scale factor in terms of tau
        alpha_a_gauss = Vexp.alpha(tau)
        sf_a_gauss.append(np.exp(alpha_a_gauss) / Vexp.props.xb)
        # Power spectra
        PB, PE, J11, J22 = csq1d_gauss.eval_PB_PE(Vexp, tau)
        PB_a_gauss.append(PB)
        PE_a_gauss.append(PE)

    for tau in tau_evol_a_cauchy:
        # Scale factor in terms of tau
        alpha_a_cauchy = Vexp.alpha(tau)
        sf_a_cauchy.append(np.exp(alpha_a_cauchy) / Vexp.props.xb)
        # Power spectra
        PB, PE, J11, J22 = csq1d_cauchy.eval_PB_PE(Vexp, tau)
        PB_a_cauchy.append(PB)
        PE_a_cauchy.append(PE)

    PB_a_nopot = np.array(PB_a_nopot)
    PE_a_nopot = np.array(PE_a_nopot)
    sf_a_nopot = np.array(sf_a_nopot)

    rho_phi_a = np.array(rho_phi_a)

    PB_a_gauss = np.array(PB_a_gauss)
    PE_a_gauss = np.array(PE_a_gauss)
    sf_a_gauss = np.array(sf_a_gauss)

    PB_a_cauchy = np.array(PB_a_cauchy)
    PE_a_cauchy = np.array(PE_a_cauchy)
    sf_a_cauchy = np.array(sf_a_cauchy)

    c_nopot = next(color_nopot)
    c_gauss = next(color_gauss)
    c_cauchy = next(color_cauchy)

    # Scalar field
    plt.plot(tau_evol_a_nopot, rho_phi_a, c="red", lw=mylw, label=r"$\rho_{{\phi}}$")

    # EM
    plt.plot(
        tau_evol_a_nopot,
        ((PB_a_nopot + PE_a_nopot)[-1]) * sf_a_nopot[-1] ** 4 / sf_a_nopot**4,
        c=c_nopot,
        lw=mylw,
        label=r"$\rho_r^{{nopot}}$ (k = $%s$)" % latex_float(k),
    )
    plt.plot(
        tau_evol_a_gauss,
        ((PB_a_gauss + PE_a_gauss)[-1]) * sf_a_gauss[-1] ** 4 / sf_a_gauss**4,
        c=c_gauss,
        lw=mylw,
        label=r"$\rho_{{r}}^{{gauss}}$ (k = $%s$)" % latex_float(k),
    )
    plt.plot(
        tau_evol_a_cauchy,
        ((PB_a_cauchy + PE_a_cauchy)[-1]) * sf_a_cauchy[-1] ** 4 / sf_a_cauchy**4,
        c=c_cauchy,
        lw=mylw,
        label=r"$\rho_{{r}}^{{cauchy}}$ (k = $%s$)" % latex_float(k),
    )


plt.xlabel(r"$\tau$")
plt.yscale("log")
leg = plt.legend(loc="best", ncol=2, fontsize=16)
# plt.xlim(-1.0,1.0)
# plt.ylim(10e-58,1)
plt.savefig(plotpath + "plotrho_" + current_set + ".pdf")

plt.show()